In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [2]:
# Import our input dataset
athletes_df = pd.read_csv('Resources/Olympic Project.csv')
athletes_df = athletes_df.rename(columns={"ID" : "CompetitorID"})
athletes_df

,CompetitorID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [3]:
athletes_df.describe()

,CompetitorID,Age,Height,Weight,Year
count,271116.000000,261642.000000,210945.000000,208241.000000,271116.000000
mean,68248.954396,25.556898,175.338970,70.702393,1978.378480
std,39022.286345,6.393561,10.518462,14.348020,29.877632
min,1.000000,10.000000,127.000000,25.000000,1896.000000
25%,34643.000000,21.000000,168.000000,60.000000,1960.000000
50%,68205.000000,24.000000,175.000000,70.000000,1988.000000
75%,102097.250000,28.000000,183.000000,79.000000,2002.000000
max,135571.000000,97.000000,226.000000,214.000000,2016.000000


In [4]:
athletes_df = pd.get_dummies(athletes_df, columns=['Medal'])

In [5]:
list(athletes_df.columns)

['CompetitorID',
 'Name',
 'Sex',
 'Age',
 'Height',
 'Weight',
 'Team',
 'NOC',
 'Games',
 'Year',
 'Season',
 'City',
 'Sport',
 'Event',
 'Medal_Bronze',
 'Medal_Gold',
 'Medal_Silver']

In [6]:
# Create new athlete dataframe
athlete_count_df = athletes_df[['CompetitorID', 'Name', 'NOC', 'Year', 'Season', 'Sport', 'Event', 'Medal_Bronze', 'Medal_Silver', 'Medal_Gold']]

In [7]:
# Select all sports that are team only events
team_sports_df = athlete_count_df[athlete_count_df['Sport'].astype(str).str.contains("Baseball|Beach Volleyball|Bobsleigh|Cricket|Football|Handball|Hockey|Ice Hockey|Lacrosse|Polo|Rugby|Rugby Sevens|Softball|Synchronized Swimming|Tug-Of-War|Volleyball")]

In [8]:
# Drop all team members
team_sports_dropped_df = team_sports_df.drop_duplicates(subset=['NOC', 'Year', 'Sport', 'Event'])

In [9]:
# Remove all members of a team from the athletes dataframe
athlete_countv1_df = pd.merge(athlete_count_df, team_sports_df, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [10]:
# Select all members of teams that are not obvious team sports
teams_df = athlete_count_df[athlete_count_df['Event'].astype(str).str.contains("Team|Relay|Doubles|Two-Man|Fours|Synchronized|Two Person")]

In [11]:
# Drop all team members from teams df
teams_dropped_df = teams_df.drop_duplicates(subset=['NOC', 'Year', 'Sport', 'Event'])

In [12]:
# Remove all members of a team from sports that are not obvious team sports from the athletes dataframe
athlete_countv2_df = pd.merge(athlete_countv1_df, teams_df, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [13]:
# Concat athlete_countv2_df with team_sports_dropped_df and teams_dropped_df to add back in each team dropped from the dataset
dataframes = [athlete_countv2_df, team_sports_dropped_df, teams_dropped_df]
athlete_count_clean_df = pd.concat(dataframes)

In [14]:
athlete_count_df = athlete_count_clean_df[['CompetitorID', 'NOC', 'Year', 'Season']]

In [15]:
# Summer olympic participation count across all years
summer_count = athlete_count_clean_df[athlete_count_clean_df['Season'] == 'Summer'].groupby(['NOC', 'Year']).size()
summer_count_df = pd.DataFrame(summer_count, columns=['summer_athlete_count'])

In [16]:
# Winter olympic participation count across all years
winter_count = athlete_count_clean_df[athlete_count_clean_df['Season'] == 'Winter'].groupby(['NOC', 'Year']).size()
winter_count_df = pd.DataFrame(winter_count, columns=['winter_athlete_count'])

In [17]:
# Summer olympic medal counts
summer_medal_count = athlete_count_clean_df[athlete_count_clean_df['Season'] == 'Summer'].groupby(['NOC', 'Year'])['Medal_Bronze', 'Medal_Silver', 'Medal_Gold'].sum()
summer_medal_count_df = pd.DataFrame(summer_medal_count, columns=['Medal_Bronze', 'Medal_Silver', 'Medal_Gold'])

C:\Users\benja\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [18]:
# Winter olympic medal counts
winter_medal_count = athlete_count_clean_df[athlete_count_clean_df['Season'] == 'Winter'].groupby(['NOC', 'Year'])['Medal_Bronze', 'Medal_Silver', 'Medal_Gold'].sum()
winter_medal_count_df = pd.DataFrame(winter_medal_count, columns=['Medal_Bronze', 'Medal_Silver', 'Medal_Gold'])

C:\Users\benja\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [21]:
# Set up host dataframe
host_df = pd.read_csv('Resources/Host_Country.csv')
host_df = host_df.drop(['Country', 'Continent', 'City'], axis=1)
host_df['Host'] = 1
host_df

,NOC,Year,Summer,Winter,Host
0,GRE,1896,I,NaN,1
1,FRA,1900,II,NaN,1
2,USA,1904,III,NaN,1
3,GBR,1908,IV,NaN,1
4,SWE,1912,V,NaN,1
5,BEL,1920,VII,NaN,1
6,FRA,1924,NaN,I,1
7,FRA,1924,VIII,NaN,1
8,SUI,1928,NaN,II,1
9,NED,1928,IX,NaN,1


In [31]:
# Summer and Winter Host
summer_host_df = host_df[['NOC', 'Year', 'Summer', 'Host']].dropna()
summer_host_df['Season'] = 'Summer'
summer_host_df = summer_host_df.drop(['Summer'], axis=1)
winter_host_df = host_df[['NOC', 'Year', 'Winter', 'Host']].dropna()
winter_host_df['Season'] = 'Winter'
winter_host_df = winter_host_df.drop(['Winter'],axis=1)
frames = [summer_host_df, winter_host_df]
clean_host_df = pd.concat(frames)
clean_host_df

,NOC,Year,Host,Season
0,GRE,1896,1,Summer
1,FRA,1900,1,Summer
2,USA,1904,1,Summer
3,GBR,1908,1,Summer
4,SWE,1912,1,Summer
5,BEL,1920,1,Summer
7,FRA,1924,1,Summer
9,NED,1928,1,Summer
11,USA,1932,1,Summer
13,GER,1936,1,Summer


In [32]:
# Individual athlete count with host dataframe
final_athlete_count_df = pd.merge(athlete_count_df, clean_host_df, how='left', left_on=['NOC', 'Year', 'Season'], right_on=['NOC', 'Year', 'Season'])
final_athlete_count_df['Host'] = final_athlete_count_df['Host'].fillna(0)
final_athlete_count_df['Host'] = final_athlete_count_df['Host'].astype(int)
final_athlete_count_df = final_athlete_count_df.reset_index(drop=True)
final_athlete_count_df

,CompetitorID,NOC,Year,Season,Host
0,1,CHN,1992,Summer,0
1,2,CHN,2012,Summer,0
2,5,NED,1988,Winter,0
3,5,NED,1988,Winter,0
4,5,NED,1992,Winter,0
...,...,...,...,...,...
203355,132824,ISR,2000,Summer,0
203356,133378,CHN,2008,Summer,1
203357,133923,ITA,2012,Summer,0
203358,134322,BOH,1906,Summer,0


In [33]:
# Athlete count by country 
count_by_country_year = final_athlete_count_df.groupby(['NOC', 'Year', 'Season', 'Host'])['CompetitorID'].count()
count_by_country_year_df = pd.DataFrame(count_by_country_year)
count_by_country_year_df = count_by_country_year_df.reset_index()
count_by_country_year_df.head()

,NOC,Year,Season,Host,CompetitorID
0,AFG,1936,Summer,0,4
1,AFG,1948,Summer,0,2
2,AFG,1956,Summer,0,1
3,AFG,1960,Summer,0,13
4,AFG,1964,Summer,0,8


In [34]:
# Country athlete counts to csv
count_by_country_year_df.to_csv('CSV_In_Progress/competitor_count.csv')

In [ ]:
# Individual Athlete count to csv
final_athlete_count_df.to_csv('CSV_In_Progress/athlete_count_final.csv')